In [1]:
#pip install nltk

In [2]:
#pip install wordcloud

In [3]:
#pip install textblob

In [4]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
import re
from nltk.stem import WordNetLemmatizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from textblob import TextBlob
from textblob import Word
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

import warnings
warnings.filterwarnings('ignore')

In [5]:
columns = ['id','source','type','text']
df = pd.read_csv('twitter_training.csv',names=columns )
df.head()

,id,source,type,text
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...


In [6]:
df.drop_duplicates()

,id,source,type,text
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
...,...,...,...,...
74677,9200,Nvidia,Positive,Just realized that the Windows partition of my...
74678,9200,Nvidia,Positive,Just realized that my Mac window partition is ...
74679,9200,Nvidia,Positive,Just realized the windows partition of my Mac ...
74680,9200,Nvidia,Positive,Just realized between the windows partition of...


In [7]:
df['type'].value_counts()

type
Negative      22542
Positive      20832
Neutral       18318
Irrelevant    12990
Name: count, dtype: int64

In [8]:
df['source'].value_counts()

source
TomClancysRainbowSix                 2400
MaddenNFL                            2400
Microsoft                            2400
LeagueOfLegends                      2394
CallOfDuty                           2394
Verizon                              2382
CallOfDutyBlackopsColdWar            2376
ApexLegends                          2376
Facebook                             2370
WorldOfCraft                         2364
Dota2                                2364
NBA2K                                2352
TomClancysGhostRecon                 2346
Battlefield                          2346
FIFA                                 2340
Xbox(Xseries)                        2334
Overwatch                            2334
johnson&johnson                      2328
Amazon                               2316
PlayStation5(PS5)                    2310
HomeDepot                            2310
Cyberpunk2077                        2304
CS-GO                                2304
GrandTheftAuto(GTA)        

In [9]:
df['source'].nunique()

32

In [10]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [11]:
def clean_text(text):
    if isinstance(text, str):  # Check if the value is a string
        # Convert text to lowercase
        text = text.lower()
        
        # Remove special characters, hyperlinks, and usernames
        text = re.sub(r"http\S+|www\S+|@[^\s]+|\n|\r", "", text)
        text = re.sub(r"[^\w\s]", "", text)
        
        # Tokenize text
        tokens = word_tokenize(text)
        
        # Remove stopwords and non-alphabetic tokens
        stop_words = set(stopwords.words('english'))
        tokens = [token for token in tokens if token.isalpha() and token not in stop_words]
        
        # Lemmatize tokens
        lemmatizer = WordNetLemmatizer()
        tokens = [lemmatizer.lemmatize(token) for token in tokens]
        
        # Join tokens back into a cleaned text
        cleaned_text = ' '.join(tokens)
        return cleaned_text
    else:
        return ''  # Return empty string for non-string values


In [12]:
df.dtypes

id         int64
source    object
type      object
text      object
dtype: object

In [13]:
df['cleaned_tweet'] = df['text'].apply(clean_text)

In [14]:
df.head()

,id,source,type,text,cleaned_tweet
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...,im getting borderland murder
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...,coming border kill
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...,im getting borderland kill
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...,im coming borderland murder
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...,im getting borderland murder


In [15]:
df = df.dropna()

In [16]:
from sklearn.feature_extraction.text import TfidfTransformer

In [23]:
tweets = df['cleaned_tweet']

In [24]:

# Initialize TfidfTransformer to create TF-IDF representation from BoW
tfidf_transformer = TfidfTransformer()

count_vectorizer = CountVectorizer(max_features=5000)  # Limit to top 5000 features
bow_matrix = count_vectorizer.fit_transform(tweets)

# Fit and transform the BoW matrix to generate TF-IDF representation
tfidf_matrix = tfidf_transformer.fit_transform(bow_matrix)

# Get the feature names (words)
feature_names = count_vectorizer.get_feature_names_out()

# Create a DataFrame to display the TF-IDF representation
tfidf_df = pd.DataFrame.sparse.from_spmatrix(tfidf_matrix, columns=feature_names)

# Displaying the TF-IDF DataFrame
print("TF-IDF representation:")
print(tfidf_df.head())

TF-IDF representation:
    aa  aaa   ab  abandon  abandoned  abc  ability  able  absolute  \
0  0.0  0.0  0.0      0.0        0.0  0.0      0.0   0.0       0.0   
1  0.0  0.0  0.0      0.0        0.0  0.0      0.0   0.0       0.0   
2  0.0  0.0  0.0      0.0        0.0  0.0      0.0   0.0       0.0   
3  0.0  0.0  0.0      0.0        0.0  0.0      0.0   0.0       0.0   
4  0.0  0.0  0.0      0.0        0.0  0.0      0.0   0.0       0.0   

   absolutely  ...  zelda  zen  zero  zip  zoe  zombie  zone  zonestreamcx  \
0         0.0  ...    0.0  0.0   0.0  0.0  0.0     0.0   0.0           0.0   
1         0.0  ...    0.0  0.0   0.0  0.0  0.0     0.0   0.0           0.0   
2         0.0  ...    0.0  0.0   0.0  0.0  0.0     0.0   0.0           0.0   
3         0.0  ...    0.0  0.0   0.0  0.0  0.0     0.0   0.0           0.0   
4         0.0  ...    0.0  0.0   0.0  0.0  0.0     0.0   0.0           0.0   

   zoom  zuckerberg  
0   0.0         0.0  
1   0.0         0.0  
2   0.0         0.0  

In [25]:
from sklearn.model_selection import train_test_split

# Assuming tfidf_matrix is your TF-IDF representation and data['sentiment'] contains labels

# Splitting TF-IDF matrix and labels into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(tfidf_matrix, df['type'], test_size=0.2, random_state=42)

# Displaying the shapes of the splits
print("Training set - X:", X_train.shape, " y:", y_train.shape)
print("Testing set - X:", X_test.shape, " y:", y_test.shape)


Training set - X: (59196, 5000)  y: (59196,)
Testing set - X: (14800, 5000)  y: (14800,)


In [26]:
# Initialize Logistic Regression model
logistic_model = LogisticRegression(max_iter=1000)  # Increase max_iter if necessary

# Train the model on the training data
logistic_model.fit(X_train, y_train)

# Make predictions on the test data
predictions = logistic_model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, predictions)
print("Accuracy:", accuracy)

# Display classification report
print("Classification Report:")
print(classification_report(y_test, predictions))

Accuracy: 0.6757432432432432
Classification Report:
              precision    recall  f1-score   support

  Irrelevant       0.70      0.49      0.58      2696
    Negative       0.67      0.80      0.73      4380
     Neutral       0.67      0.61      0.64      3605
    Positive       0.68      0.73      0.70      4119

    accuracy                           0.68     14800
   macro avg       0.68      0.66      0.66     14800
weighted avg       0.68      0.68      0.67     14800



In [27]:
from sklearn.naive_bayes import GaussianNB

In [28]:
# Initialize Gaussian Naive Bayes classifier
naive_bayes = GaussianNB()

# Train the model on the training data
naive_bayes.fit(X_train.toarray(), y_train)  # Convert sparse matrix to array for GaussianNB

# Make predictions on the test data
predictions = naive_bayes.predict(X_test.toarray())  # Convert sparse matrix to array for predictions

# Calculate accuracy
accuracy = accuracy_score(y_test, predictions)
print("Accuracy:", accuracy)

# Display classification report
print("Classification Report:")
print(classification_report(y_test, predictions))

Accuracy: 0.4250675675675676
Classification Report:
              precision    recall  f1-score   support

  Irrelevant       0.28      0.93      0.43      2696
    Negative       0.75      0.30      0.43      4380
     Neutral       0.76      0.25      0.37      3605
    Positive       0.52      0.38      0.44      4119

    accuracy                           0.43     14800
   macro avg       0.58      0.46      0.42     14800
weighted avg       0.60      0.43      0.42     14800



In [34]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, SpatialDropout1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

# Assuming you have text data in 'data['tweet content']' and labels in 'data['sentiment']'

# Tokenize and pad sequences
max_features = 10000  # Maximum number of words to consider
max_len = 100  # Maximum length of sequences
tokenizer = Tokenizer(num_words=max_features, oov_token='<OOV>')
tokenizer.fit_on_texts(df['cleaned_tweet'])
sequences = tokenizer.texts_to_sequences(df['type'])
X = pad_sequences(sequences, maxlen=max_len)
y = df['type']


# Splitting into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

from sklearn.preprocessing import LabelEncoder

# Assuming 'y_train' and 'y_test' are your string labels
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)


# Building RNN model
model = Sequential()
model.add(Embedding(max_features, 128, input_length=max_len))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Training the RNN model
model.fit(X_train, y_train_encoded, epochs=5, batch_size=32, validation_data=(X_test, y_test_encoded))

# Evaluate on test data
loss, accuracy = model.evaluate(X_test, y_test_encoded)
print("Accuracy:", accuracy)


Epoch 1/5
1850/1850 [==============================] - 316s 169ms/step - loss: 0.0000e+00 - accuracy: 0.4543 - val_loss: 0.0000e+00 - val_accuracy: 0.4781
Epoch 2/5
1850/1850 [==============================] - 328s 177ms/step - loss: 0.0000e+00 - accuracy: 0.4757 - val_loss: 0.0000e+00 - val_accuracy: 0.4781
Epoch 3/5
1850/1850 [==============================] - 328s 177ms/step - loss: 0.0000e+00 - accuracy: 0.4757 - val_loss: 0.0000e+00 - val_accuracy: 0.4781
Epoch 4/5
1850/1850 [==============================] - 342s 185ms/step - loss: 0.0000e+00 - accuracy: 0.4757 - val_loss: 0.0000e+00 - val_accuracy: 0.4781
Epoch 5/5
463/463 [==============================] - 13s 27ms/step - loss: 0.0000e+00 - accuracy: 0.4781
Accuracy: 0.47810810804367065


In [36]:
# Building LSTM model
model = Sequential()
model.add(Embedding(max_features, 128, input_length=max_len))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(128, return_sequences=True))
model.add(LSTM(64, return_sequences=True))
model.add(LSTM(32, return_sequences=True))
model.add(LSTM(16))  # Last layer without return_sequences=True
model.add(Dense(1, activation='softmax'))  # Output layer for classification

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Training the LSTM model
model.fit(X_train, y_train_encoded, epochs=5, batch_size=32, validation_data=(X_test, y_test_encoded))

# Evaluate on test data
loss, accuracy = model.evaluate(X_test, y_test_encoded)
print("Accuracy:", accuracy)

Epoch 1/5
1850/1850 [==============================] - 567s 302ms/step - loss: 0.0000e+00 - accuracy: 0.3037 - val_loss: 0.0000e+00 - val_accuracy: 0.2959
Epoch 2/5
1850/1850 [==============================] - 542s 293ms/step - loss: 0.0000e+00 - accuracy: 0.3037 - val_loss: 0.0000e+00 - val_accuracy: 0.2959
Epoch 3/5
1850/1850 [==============================] - 539s 291ms/step - loss: 0.0000e+00 - accuracy: 0.3037 - val_loss: 0.0000e+00 - val_accuracy: 0.2959
Epoch 4/5
1850/1850 [==============================] - 920s 497ms/step - loss: 0.0000e+00 - accuracy: 0.3037 - val_loss: 0.0000e+00 - val_accuracy: 0.2959
Epoch 5/5
463/463 [==============================] - 42s 91ms/step - loss: 0.0000e+00 - accuracy: 0.2959
Accuracy: 0.295945942401886
